#### Initial Setup

In [1]:
import os

In [2]:
%pwd

'E:\\RajaRajeshwari\\MyFolders\\Projects\\mlops_water_potability_prediction\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'E:\\RajaRajeshwari\\MyFolders\\Projects\\mlops_water_potability_prediction'

In [5]:
import pandas as pd

df = pd.read_csv("artifacts/data_ingestion/water_potability_dataset.csv")

#### Entity

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_path: Path
    status_file: str
    data_schema: dict

#### Configuration

In [7]:
from src.mlops_water_potability_prediction_project.constants import *
from src.mlops_water_potability_prediction_project.utilities.helpers import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories(directories_path_list=[self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            unzip_data_path=config.unzip_data_path,
            status_file=config.status_file,
            data_schema=schema
        )

        return data_validation_config

#### Component

In [8]:
import os
import urllib.request as request
import zipfile
from src.mlops_water_potability_prediction_project import logger
from src.mlops_water_potability_prediction_project.utilities.helpers import get_file_size

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None
            dataframe = pd.read_csv(self.config.unzip_data_path)
            all_columns = list(dataframe.columns)
            data_schema_keys = self.config.data_schema.keys()

            for col in all_columns:
                if col not in data_schema_keys:
                    validation_status = False
                elif dataframe[col].dtype != self.config.data_schema[col]:
                    validation_status = False
                else:
                    validation_status = True

            with open(self.config.status_file, 'w') as f:
                f.write(f"Validation status: {str(validation_status).upper()}")

            return validation_status
        except Exception as e:
            raise e

#### Pipeline

In [9]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-01-28 20:17:24,228]: INFO: helpers: YAML file: config\config.yaml loaded successfully]
[2024-01-28 20:17:24,230]: INFO: helpers: YAML file: params.yaml loaded successfully]
[2024-01-28 20:17:24,231]: INFO: helpers: YAML file: schema.yaml loaded successfully]
[2024-01-28 20:17:24,232]: INFO: helpers: Created directory at: artifacts]
[2024-01-28 20:17:24,233]: INFO: helpers: Created directory at: artifacts/data_validation]
